In [1]:
from processing.sentiment import GCSSentimentAnalyzer

In [2]:
import pandas as pd
from google.cloud import storage

import pandas as pd
from io import BytesIO
from google.cloud import bigquery
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.preprocessing import RobustScaler


In [3]:
import logging
bucket_name = 'ccai-storage'
file_path = 'output_tokenized.parquet'
folder = 'make_prediction'
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s\n')

output_processing = 'test_sentiment.parquet'
text_column = 'body_pre'
num_doc=10
processor=GCSSentimentAnalyzer(bucket_name=bucket_name, file_path=file_path, 
                               folder=folder, parquet_file_name=output_processing, 
                 text_column= text_column, num_doc=num_doc)
processor.process()


INFO: Check if the file in exists in gcs test_sentiment.parquet

INFO: Start loading DataFrame from output_tokenized.parquet

INFO: Loaded DataFrame from output_tokenized.parquet

INFO: Extracting Sentiment



Check if the file in exists in gcs
Start loading DataFrame from output_tokenized.parquet
Loaded DataFrame from output_tokenized.parquet
Extracting Sentiment


INFO: Sentiment extracted - process completed and file saved to GCS.



Sentiment extracted - process completed and file saved to GCS.


'test_sentiment.parquet'

In [4]:

# Your GCS bucket name
bucket_name = 'ccai-storage'

# Paths to the Parquet files in your GCS bucket
articles_parquet_path = f'{folder}/test_sentiment.parquet'

# Initialize the GCS client
client = storage.Client()

# Function to download a Parquet file from GCS and load it into a pandas DataFrame
def load_parquet_from_gcs(bucket_name, file_path):
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    byte_stream = BytesIO()
    blob.download_to_file(byte_stream)
    byte_stream.seek(0)
    df = pd.read_parquet(byte_stream, engine='pyarrow')
    return df

# Load the Parquet files
articles_df = load_parquet_from_gcs(bucket_name, articles_parquet_path)

In [5]:
articles_df

,uri,url,title,body,date,time,dateTime,dateTimePub,lang,isDuplicate,...,alexaCountryRank,date_column,year,month,year_month,topic,shares_scaled,body_pre,score,magnitude
246,7890787458,https://www.corriere.it/sette/opinioni/23_dice...,"Mario Sconcerti, il giornalista del calcio com...",Puntò il dito contro le società che controllav...,2023-12-17,10:12:25,2023-12-17T10:12:25Z,2023-12-17T10:11:10Z,ita,False,...,30,2023-12-17,2023,12,2023-12,Sports,-0.392857,Puntò dito società controllavano allenatori gi...,0.0,2.5
802,7894657293,https://www.corriere.it/animali/23_dicembre_19...,Lav e Vigili del Fuoco firmano un Protocollo d...,Siglato al Viminale un Protocollo storico che ...,2023-12-19,16:22:22,2023-12-19T16:22:22Z,2023-12-19T16:21:43Z,ita,False,...,30,2023-12-19,2023,12,2023-12,Politics,2.035714,Siglato Viminale Protocollo storico prevede co...,0.3,3.4
861,2023-12-197163529,https://www.lastampa.it/esteri/2023/12/19/news...,La guerra in Ucraina e i negoziati interrotti:...,Tra le fonti sulla presunta interferenza occid...,2023-12-19,11:00:20,2023-12-19T11:00:20Z,2023-12-19T10:47:05Z,ita,False,...,91,2023-12-19,2023,12,2023-12,Politics,-0.321429,fonti presunta interferenza occidentale alcuni...,-0.1,14.4
854,2023-12-197207617,https://www.fanpage.it/milano/il-supermercato-...,Il supermercato che paga 5mila euro i commessi...,Cinque settimane di ferie per tutti i dipenden...,2023-12-19,11:36:01,2023-12-19T11:36:01Z,2023-12-19T11:32:11Z,ita,False,...,68,2023-12-19,2023,12,2023-12,Business,6.250000,Cinque settimane ferie dipendenti ( apprendist...,-0.1,1.8
164,7892018573,https://www.ilfattoquotidiano.it/2023/12/18/ma...,"Manovra, nella notte il sì alle modifiche: dal...",Gli emendamenti delle opposizioni\n\nVia liber...,2023-12-18,07:59:56,2023-12-18T07:59:56Z,2023-12-18T07:57:48Z,ita,False,...,47,2023-12-18,2023,12,2023-12,Politics,0.821429,emendamenti opposizioni Via libera commissione...,0.0,3.9
291,2023-12-194771785,https://www.lastampa.it/biella/2023/12/17/news...,A Oropa un viaggio sulle note del Novecento co...,"Un viaggio tra le musiche del Novecento, affid...",2023-12-17,06:18:58,2023-12-17T06:18:58Z,2023-12-17T06:00:00Z,ita,False,...,91,2023-12-17,2023,12,2023-12,Arts and Entertainment,-0.107143,"viaggio musiche Novecento , affidato coro tori...",0.3,4.6
674,2023-12-198274670,https://www.repubblica.it/spettacoli/tv-radio/...,"Berlino e Al Bano ed è subito 'Felicità', per ...",Ieri sera bagno di folla al Pincio con Pedro A...,2023-12-20,09:14:24,2023-12-20T09:14:24Z,2023-12-20T09:02:59Z,ita,False,...,8,2023-12-20,2023,12,2023-12,Arts and Entertainment,0.107143,Ieri serum bagno folla Pincio Pedro Alonso kar...,0.2,3.7
518,2023-12-199263643,https://www.lastampa.it/cuneo/2023/12/21/news/...,Gli industriali di Cuneo lanciano una Fondazio...,"""Il lavoro è uno strumento di guarigione socia...",2023-12-21,05:26:30,2023-12-21T05:26:30Z,2023-12-21T05:00:00Z,ita,False,...,91,2023-12-21,2023,12,2023-12,Business,-0.321429,`` lavoro strumento guarigione sociale '' . co...,0.4,6.6
392,7898176095,https://www.ilfattoquotidiano.it/2023/12/21/su...,"Superlega, c'è chi dice ""no"": molte big del ca...","Quanti ""no"" piovono sulla nuova Superlega dopo...",2023-12-21,15:59:06,2023-12-21T15:59:06Z,2023-12-21T15:56:01Z,ita,False,...,47,2023-12-21,2023,12,2023-12,Sports,0.250000,`` no '' piovono nuova Superlega dopo sentenza...,0.0,3.1
188,2023-12-195607378,https://www.lastampa.it/biella/2023/12/18/news...,Biella Rugby senza problemi a Milano: continua...,"Tutto sul velluto per il Biella Rugby, che fa ...",2023-12-18,06:15:42,2023-12-18T06:15:42Z,2023-12-18T06:00:00Z,ita,False,...,91,2023-12-18,2023,12,2023-12,Business,-0.321429,"velluto Biella Rugby , fa bottino pieno punti ...",0.3,5.6
